# Clustering Zooniverse Marks to count Iguanas
The goal is to find the best method to cluster the data and find the best number of clusters.
The benchmark is a gold standard dataset obtained by experts.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./")
sys.path.append("./zooniverse")

## Intro
### Retrieve a Classification report from Zooniverse
Export the classification export from your zooniverse project.
https://www.zooniverse.org/lab/11905/data-exports

This leads to a csv file which can be used for the analysis which should be renamed to `iguanas-from-above-classifications.csv` and placed in the `input_path` directory.
The methods do not use methods from zooniverse. It is a custom implementation.

An alternative would be to use the [code provided by zooniverse](https://github.com/zooniverse/Data-digging/tree/master/notebooks_ProcessExports)
(Bird Count Example)[https://github.com/zooniverse/Data-digging/blob/master/scripts_ProjectExamples/seabirdwatch/bird_count.py]

This notebooks assumes the data is flat and prepared. An alternative format would be the [cesar aggregation format](https://github.com/zooniverse/aggregation-for-caesar)

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

### Processing workflow

The Data is flatted and filtered
![Image](images/dataprocessing-DataFiltering.png)

In each phase we have the following number of images if they are filtered for at least 4 true marks by users
1. phase 105
2. phase 160
3. phase 86


## Load the data

In [10]:
from zooniverse.utils.data_format import data_prep
from pathlib import Path

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold
from zooniverse.config import get_config

## Input Path of all the data
input_path =Path("/Users/christian/data/zooniverse")

reprocess = True # if True, the raw classification data is reprocessed. If False, the data is loaded from disk

# Phase Selection
phase_tag = "Iguanas 1st launch"
# phase_tag = "Iguanas 2nd launch"
phase_tag = "Iguanas 3rd launch"


debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook

user_threshold = 5 # None or a number, filter records which have less than these user interactions.


### use either the subset of the subset
# use_gold_standard_subset = "expert_goldstandard" # Use the X-T2-GS-results-5th-0s as the basis
use_gold_standard_subset = "expert" # Use the expert-GS-Xphase as the basis

# Location for the analysis Results
output_path = Path(input_path.joinpath(f"2024_03_07_{use_gold_standard_subset}_analysis").joinpath(phase_tag))
output_path.mkdir(exist_ok=True, parents=True)

# Location for plots
output_plot_path = output_path.joinpath("plots")
output_plot_path.mkdir(parents=True, exist_ok=True)


## Look into the config
This Config points to all files necessary for the analysis + the result files

In [11]:
config = get_config(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('/Users/christian/data/zooniverse/IguanasFromAbove/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-3rdphase_renamed.csv'),
 'gold_standard_image_subset': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/3-T2-GS-results-5th-0s.csv'),
 'image_source': None,
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_03_07_None_analysis/Iguanas 3rd launch/yes_no_dataset_Iguanas 3rd launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_03_07_None_analysis/Iguanas 3rd launch/flat_dataset_Iguanas 3rd launch.csv'),
 'merged_dataset': PosixPath('/Users/christian/data/zooniverse/2024_03_07_None_analysis/Iguanas 3rd launch/merged_dataset_gold_standard_expert_Iguanas 3rd launch_filtered.csv'),
 'gold_standard_and_expert_count': PosixPath('/Users/christian/data/zooniverse/2024_03_07

In [12]:
if reprocess:
    ds_stats = data_prep(phase_tag=phase_tag, 
                         output_path=output_path, 
                         input_path=input_path,
                         filter_combination=use_gold_standard_subset, 
                         config=config)
    print(ds_stats)

2024-03-18 16:25:24.113 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:178 - found a new phase tag: Iguanas 1st launch
2024-03-18 16:25:31.753 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:178 - found a new phase tag: Main Workflow
2024-03-18 16:25:33.224 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:178 - found a new phase tag: survey tool 061417
2024-03-18 16:25:34.867 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:178 - found a new phase tag: Iguanas 2nd launch
2024-03-18 16:25:39.134 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:178 - found a new phase tag: Iguanas 3rd launch


{'retired': None, 'name': 'PCI01-1_149.jpg', 'type': 'test'}
{'retired': None, 'name': 'PCI02-1_92.jpg', 'type': 'test'}
{'retired': None, 'name': 'PCIS02-1_40.jpg', 'type': 'test'}


2024-03-18 16:25:54.036 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:178 - found a new phase tag: Iguanas 4th launch
2024-03-18 16:25:54.036 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:178 - found a new phase tag: Plastics GS dataset
2024-03-18 16:25:54.053 | INFO     | zooniverse.utils.data_format:read_zooniverse_annotations_v2:258 - generating flat dataframe from the dataset
2024-03-18 16:25:56.896 | INFO     | zooniverse.utils.data_format:data_prep:303 - flat_dataset_Iguanas Iguanas 3rd launch.csv: 8330 images in classification for Iguanas 3rd launch
2024-03-18 16:25:56.930 | INFO     | zooniverse.utils.data_format:data_prep:325 - found 1156 images in /Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-3rdphase_renamed.csv, the expert counts
2024-03-18 16:25:56.932 | INFO     | zooniverse.utils.data_format:data_prep:333 - working with 87 images in /Users/christian/data/zooniverse/Images/Zooniverse_

                                            filename  images
0                flat_dataset_Iguanas 3rd launch.csv    8330
1                     expert-GS-3rdphase_renamed.csv    1156
2                         3-T2-GS-results-5th-0s.csv      87
3  merged_dataset_gold_standard_expert_Iguanas 3r...    7664


### Optional Debugging

In [13]:
if plot_diagrams == False:
    output_plot_path = None

# the flattened, filtered marks from zooniverse.
df_merged_dataset = pd.read_csv(config["merged_dataset"])

# data for reference
df_goldstandard_expert_count = pd.read_csv(config["goldstandard_data"], sep=";")

## Debugging helpers
if phase_tag == "Iguanas 1st launch":    
    if debug:

        df_merged_dataset = df_merged_dataset[df_merged_dataset.image_name.isin(["SFM01-2-2-2_333.jpg", "SFM01-2-2-2_334.jpg", "SFM01-2-2-3_201.jpg"])]

elif phase_tag == "Iguanas 2nd launch":
    if debug:
        df_merged_dataset = df_merged_dataset[
           df_merged_dataset.image_name.isin(["FMO03-1_65.jpg", "FMO03-1_72.jpg", "MBN04-2_182.jpg", "EGI08-2_78.jpg"])]
           # df_merged_dataset.image_name.isin(["FMO03-1_72.jpg"])]

    
elif phase_tag == "Iguanas 3rd launch":

    # this user is a spammer
    df_merged_dataset = df_merged_dataset[df_merged_dataset.user_id != 2581179]
    
    if debug:
        df_merged_dataset = df_merged_dataset[
           df_merged_dataset.image_name.isin(["FMO03-2_70.jpg", "MBN04-2_182.jpg", "EGI08-2_78.jpg"])]
            
    


## Look at the data


In [14]:
## Look at the data
df_merged_dataset.drop("user_name", axis=1)


,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id
0,0,NaN,NaN,78861920,937.072327,58.004669,Adult Male not in a lek,Iguanas 3rd launch,1983945.0
1,1,NaN,NaN,78861920,1094.908203,1027.116699,Adult Male not in a lek,Iguanas 3rd launch,1983945.0
2,2,NaN,NaN,78861920,782.393250,62.739746,"Others (females, young males, juveniles)",Iguanas 3rd launch,1983945.0
3,3,NaN,NaN,78861920,861.311157,275.818115,"Others (females, young males, juveniles)",Iguanas 3rd launch,1983945.0
4,4,NaN,NaN,78861920,982.844727,353.157684,"Others (females, young males, juveniles)",Iguanas 3rd launch,1983945.0
...,...,...,...,...,...,...,...,...,...
106332,113868,GEB02,GEB02-3_48.jpg,78922637,857.985596,933.296021,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN
106333,113869,GEB02,GEB02-3_115.jpg,78922583,600.195435,856.913757,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN
106334,113870,GEB02,GEB02-3_197.jpg,78922625,485.176636,426.866882,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN
106335,113871,GEB02,GEB02-3_145.jpg,78922607,939.968628,88.864525,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN


### Filter User if necessary and Marks


In [15]:
print(f"Before filtering: {df_merged_dataset.subject_id.nunique()}")
# There images in which some people said there are iguanas, but then didn't mark them. Clustering with fewer than 3 dots doesn't make sense
if user_threshold is not None:
    print(f"filtering records which have less than {user_threshold} interactions.")
    df_merged_dataset = filter_df_user_threshold(df_merged_dataset, user_threshold=user_threshold)
    
    
from zooniverse.utils.filters import filter_remove_marks
# Check if partials are still in the data. There shouldn't be any
df_merged_dataset = filter_remove_marks(df_merged_dataset)


2024-03-18 16:26:03.565 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image EGI01-1-2_121.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.566 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image EGI02-2_114.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.566 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image EGI02-2_55.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.567 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image EGI03_48.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.568 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image EGI04-2_106.jpg has only 2 users, which is less than the threshold of 5
2024-03-18 16:26:03.569 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image EGI09-2_175.jpg has only 1 use

Before filtering: 7580
filtering records which have less than 5 interactions.


2024-03-18 16:26:03.760 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image FMO02-1_128.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.761 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image FMO02-1_134.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.762 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image FMO02-1_138.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.762 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image FMO02-1_140.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.763 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image FMO02-1_141.jpg has only 1 users, which is less than the threshold of 5
2024-03-18 16:26:03.763 | WARNING  | zooniverse.utils.filters:filter_df_user_threshold:22 - The image FMO02-1_143.jpg has only 1 u

### Are there anonymous users in the data?
There should be

In [16]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name
101280,108395,NaN,EGI04-1_141.jpg,78963577,178.789062,142.882812,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-ba2607662ff97583013c
101281,108396,NaN,EGI04-1_141.jpg,78963577,80.406250,239.140625,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-ba2607662ff97583013c
92404,98952,NaN,EGI08-2_78.jpg,78963961,263.500549,597.307312,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-4d14529c77a0f90d34e1
32482,35456,NaN,EPS01_28.jpg,78963899,404.259186,676.611877,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-1887ca46cc6d8259af6e
38206,41612,NaN,EPS01_28.jpg,78963899,402.240112,679.267700,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-9d378135ba9a36e66685
...,...,...,...,...,...,...,...,...,...,...
10705,11577,WestCoast,PWC03-2-4_82.jpg,78963461,292.804230,1022.759766,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-d6bf6dd1bb0d970b9db0
23628,26014,WestCoast,PWC03-2-4_82.jpg,78963461,300.249969,1024.011719,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-cbb14fd93a9b4480cb6e
13624,14905,WestCoast,PWC03-2-4_85.jpg,78963464,548.868530,51.378170,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-c622f0bbbff06c84863a
44738,48532,WestCoast,PWC03-2-4_85.jpg,78963464,548.724854,55.921528,Adult Male not in a lek,Iguanas 3rd launch,NaN,not-logged-in-7be09a77297e02445aad


In [17]:
# Amount of images
df_merged_dataset["subject_id"].nunique()

1035

In [18]:
## After filtering there
df_merged_dataset

,Unnamed: 0,flight_site_code,image_name,subject_id,x,y,tool_label,phase_tag,user_id,user_name
13048,14294,NaN,EGI04-1_141.jpg,78963577,174.377457,140.835648,Adult Male not in a lek,Iguanas 3rd launch,1325881.0,kmorrisseyukyahoo.co.uk
24783,27310,NaN,EGI04-1_141.jpg,78963577,169.812500,141.750000,Adult Male not in a lek,Iguanas 3rd launch,2429803.0,MarieKatrin
25186,27760,NaN,EGI04-1_141.jpg,78963577,176.250000,143.966675,"Others (females, young males, juveniles)",Iguanas 3rd launch,2526505.0,sabrabel
25187,27761,NaN,EGI04-1_141.jpg,78963577,81.250000,238.966675,"Others (females, young males, juveniles)",Iguanas 3rd launch,2526505.0,sabrabel
29181,31981,NaN,EGI04-1_141.jpg,78963577,173.804688,139.500000,Adult Male with a lek,Iguanas 3rd launch,2541149.0,z.bee
...,...,...,...,...,...,...,...,...,...,...
66303,72154,WestCoast,PWC03-2-5_114.jpg,78963477,24.796158,602.070312,"Others (females, young males, juveniles)",Iguanas 3rd launch,2574350.0,mperky2
92664,99222,WestCoast,PWC03-2-5_114.jpg,78963477,23.494946,604.926392,"Others (females, young males, juveniles)",Iguanas 3rd launch,NaN,not-logged-in-2c607f3a99d9a7851873
92892,99476,WestCoast,PWC03-2-5_114.jpg,78963477,25.454941,601.314026,"Others (females, young males, juveniles)",Iguanas 3rd launch,2309767.0,laswett
98013,104954,WestCoast,PWC03-2-5_114.jpg,78963477,26.607056,604.818420,"Others (females, young males, juveniles)",Iguanas 3rd launch,1912057.0,ATX


In [19]:
# how many marks per user
df_merged_dataset[["user_id", "x"]].groupby("user_id").count().head()

,x
user_id,
42.0,27
1476.0,1
2842.0,15
3427.0,28
3885.0,4


### gold standard data
For reference

In [20]:
df_goldstandard_expert_count[df_goldstandard_expert_count["image_name"].isin(["SFM01-2-2-2_282.jpg", "SFM01-2-2-2_323.jpg"])]

,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment


In [21]:
df_goldstandard_expert_count.count_total.sum()

388

In [22]:
# look at the
df_goldstandard_expert_count.count()

subspecies           1156
island               1156
site_name            1156
subject_group        1156
image_name           1156
subject_id           1156
presence_absence     1156
count_male-lek       1156
count_male-no-lek    1156
count_others         1156
count_partial        1156
count_total          1156
quality               117
condition             116
comment                21
dtype: int64

In [23]:
fsum = df_goldstandard_expert_count[
    df_goldstandard_expert_count.image_name.isin(df_merged_dataset.image_name.unique())]

print(f"filtering the zooniverse classifications dataset for gold standard images the count_total of iguanas is: {fsum.count_total.sum()}, but it should be {df_goldstandard_expert_count.count_total.sum()}")
fsum


filtering the zooniverse classifications dataset for gold standard images the count_total of iguanas is: 350, but it should be 388


,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment
22,A. c. hayampi,Marchena,BahiaNegra,MBN1,MBN04-2_182.jpg,78926344,N,0,0,0,0,0,NaN,NaN,NaN
100,A. c. hayampi,Marchena,BahiaNegraD,MBBD1,MBBD02-2_248.jpg,78928708,Y,0,1,1,0,2,Good,NaN,NaN
208,A. c. sielmanni,Pinta,CaboIbebetson,PCI,PCI01-2_111.jpg,78938221,Y,1,0,3,0,4,Good,Visible,NaN
238,A. c. sielmanni,Pinta,CaboIbebetson,PCI,PCI02-1_92.jpg,78938603,Y,0,1,0,0,1,Good,Visible,NaN
274,A. c. sielmanni,Pinta,CaboIbebetsonC,PCIC,PCIC01-1_113.jpg,78938992,Y,0,0,2,0,2,Good,Hard,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-2_300.jpg,78925388,Y,0,0,1,0,1,Good,Visible,NaN
1135,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-2_421.jpg,78925457,Y,0,0,1,0,1,Good,Visible,NaN
1136,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-2_430.jpg,78925467,Y,0,0,1,0,1,Good,Visible,NaN
1144,A. c. nanus,Genovesa,WestCoastB,GWB,GWB01-3_12.jpg,78925536,Y,0,0,1,0,1,Good,Visible,NaN


In [24]:
# How many images are left in the zooniverse dataset?
len(list(df_merged_dataset.image_name.unique()))

1035

In [26]:
#Is there an image in the goldstandard, which is not in the classifcations?
print(f"images in df_goldstandard_expert_count but not in df_merged_dataset: {len(set(df_goldstandard_expert_count.subject_id) - set(df_merged_dataset.subject_id.unique()))}")


df_goldstandard_expert_count.count_total.sum()
# df_merged_dataset[df_merged_dataset.image_name.isin(["SRL01-1-2_105.jpg"])]

images in df_goldstandard_expert_count but not in df_merged_dataset: 1073


388

In [73]:
df_goldstandard_expert_count[df_goldstandard_expert_count.image_name.isin(["SRL01-1-2_105.jpg"])]

,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment
1070,A. c. mertensi,San Cristobal,Loberia,SRL1,SRL01-1-2_105.jpg,47979415,N,0,0,0,0,0,NaN,NaN,NaN


In [28]:
T2_GS_results_5th_0s = pd.read_csv(config["gold_standard_image_subset"], sep=";")
T2_GS_results_5th_0s

,subject_id,Median0s,Mean0s,Max0s,Std0s,Median0s.r,Mean0s.r,Mode0s
0,78922029,2.0,1.928571,5,0.997249,2,2,2
1,78922093,9.0,11.500000,32,10.406729,9,12,9
2,78922433,1.0,1.000000,1,0.000000,1,1,1
3,78922625,1.0,1.095238,3,0.436436,1,1,1
4,78924089,2.0,1.750000,2,0.462910,2,2,2
...,...,...,...,...,...,...,...,...
82,78965032,2.0,1.964286,4,0.881167,2,2,2
83,78965058,5.0,5.620690,12,1.449478,5,6,5
84,78965066,2.0,2.272727,4,0.631085,2,2,2
85,78965103,7.0,7.464286,13,2.411118,7,7,5


## The gold standard vs. the expert count

In [75]:
# Double checking for the counts
gstd_5th = df_goldstandard_expert_count[df_goldstandard_expert_count.subject_id.isin(T2_GS_results_5th_0s.subject_id)].count_total.sum()
print(f"If the expert count ({config['goldstandard_data']})  is filtered for the subject ids in {config['gold_standard_image_subset']} the count_total is {gstd_5th} iguanas")

If the expert count (/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-1stphase.csv)  is filtered for the subject ids in /Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/1-T2-GS-results-5th-0s.csv the count_total is 331 iguanas


In [76]:
len(set(T2_GS_results_5th_0s.subject_id.unique()) - set(df_goldstandard_expert_count.subject_id))
df_goldstandard_expert_count["count_total"].sum()

422

In [77]:
df_merged_dataset["subject_id"].nunique()

976

In [78]:

df_goldstandard_expert_count = df_goldstandard_expert_count[
    df_goldstandard_expert_count.subject_id.isin(df_merged_dataset.subject_id.unique())]
df_goldstandard_expert_count = df_goldstandard_expert_count[["image_name", "subject_id", "count_total"]]

df_goldstandard_expert_count["count_total"].sum()

411

In [79]:
## plot some of the marks
from zooniverse.utils.plotting import plot_zooniverse_user_marks_v2
# FMO03-1_65
# EIG05-1_83.jpg # phase 
# MBN04-2_182.jpg # phase 3
# df_merged_dataset_filtered = df_merged_dataset[df_merged_dataset.image_name.isin(["ESCG02-1_19.jpg"])]
if phase_tag in["Iguanas 1st launch", "Iguanas 2nd launch"]  and  ( plot_diagrams or show_plots ) :
    for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
        
        ## plot the marks
        markers_plot_path = plot_zooniverse_user_marks_v2(df_image_name,
                                                          image_path=df_image_name.iloc[0]["image_path"],
                                                          image_name=image_name,
                                                          output_path=output_plot_path, show=show_plots, title=f"Markers for {image_name}", fig_size=(5,5))
        

## Clustering

### Basic Statics like mean, median, mode

In [29]:
from sklearn.metrics import mean_squared_error
from zooniverse.analysis import get_mark_overview

basic_stats = []
kmeans_knee_stats = []
kmeans_silouettes = []
mse_errors = {}


for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)

    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])

    ### basic statistics like mean, median
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    

## join the gold standard data to the basic stats
if use_gold_standard_subset is not None:
    df_comparison = df_goldstandard_expert_count.merge(df_basic_stats, on='image_name', how='left')
else:
    df_comparison = df_basic_stats


In [30]:
df_basic_stats

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count
0,EGI04-1_141.jpg,2.0,1.80,1,15,27,"[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4]"
1,EGI08-2_78.jpg,1.0,1.00,1,8,8,"[1, 1, 1, 1, 1, 1, 1, 1]"
2,EPS01_28.jpg,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,EPS01_41.jpg,2.0,1.89,2,19,36,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,EPS02_208.jpg,2.0,2.59,2,27,70,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...
1030,PWC03-2-2_92.jpg,3.5,3.12,4,16,50,"[1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]"
1031,PWC03-2-4_66.jpg,2.0,1.85,2,13,24,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4]"
1032,PWC03-2-4_82.jpg,1.0,1.00,1,16,16,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1033,PWC03-2-4_85.jpg,1.0,1.25,1,12,15,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]"


In [31]:
# There might be records with too few annotations
df_comparison[(df_comparison.count_total > 0) & (df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

AttributeError: 'DataFrame' object has no attribute 'count_total'

In [32]:
# images with an expert count of more than 0 and less than 5 different users
df_comparison[(df_comparison.count_total > 0) & (df_comparison.users < 5)].sort_values(by="users", ascending=False)


AttributeError: 'DataFrame' object has no attribute 'count_total'

In [ ]:
df_comparison["count_total"].sum()

### Fill NaN values with 0 because the errors can't be calculated otherwise

In [33]:

## Fill NaN values with 0 because the errors can't be calculated otherwise
df_comparison.fillna(0, inplace=True)


In [34]:

mse_errors["median_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.median_count,
                                                     squared=False)
mse_errors["mean_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.mean_count, squared=False)
mse_errors["mode_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.mode_count, squared=False)

pd.Series(mse_errors)

AttributeError: 'DataFrame' object has no attribute 'count_total'

In [35]:
df_comparison

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count
0,EGI04-1_141.jpg,2.0,1.80,1,15,27,"[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4]"
1,EGI08-2_78.jpg,1.0,1.00,1,8,8,"[1, 1, 1, 1, 1, 1, 1, 1]"
2,EPS01_28.jpg,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,EPS01_41.jpg,2.0,1.89,2,19,36,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,EPS02_208.jpg,2.0,2.59,2,27,70,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...
1030,PWC03-2-2_92.jpg,3.5,3.12,4,16,50,"[1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]"
1031,PWC03-2-4_66.jpg,2.0,1.85,2,13,24,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4]"
1032,PWC03-2-4_82.jpg,1.0,1.00,1,16,16,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1033,PWC03-2-4_85.jpg,1.0,1.25,1,12,15,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]"


In [36]:

pd.Series(mse_errors)

Series([], dtype: object)

In [37]:
df_comparison

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count
0,EGI04-1_141.jpg,2.0,1.80,1,15,27,"[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4]"
1,EGI08-2_78.jpg,1.0,1.00,1,8,8,"[1, 1, 1, 1, 1, 1, 1, 1]"
2,EPS01_28.jpg,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,EPS01_41.jpg,2.0,1.89,2,19,36,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,EPS02_208.jpg,2.0,2.59,2,27,70,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...
1030,PWC03-2-2_92.jpg,3.5,3.12,4,16,50,"[1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]"
1031,PWC03-2-4_66.jpg,2.0,1.85,2,13,24,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4]"
1032,PWC03-2-4_82.jpg,1.0,1.00,1,16,16,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1033,PWC03-2-4_85.jpg,1.0,1.25,1,12,15,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]"


In [38]:
df_comparison.sum()

image_name               EGI04-1_141.jpgEGI08-2_78.jpgEPS01_28.jpgEPS01...
median_count                                                        3374.0
mean_count                                                         3513.25
mode_count                                                            3331
users                                                                17480
sum_annotations_count                                                67635
annotations_count        [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4, ...
dtype: object

### DBSCAN clustering and take the variant with the best silouette score for each image


In [39]:
### The old variant
# from zooniverse.analysis import compare_dbscan_hyp_v2
# 
# eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
# min_samples_variants = [3, 5, 8, 10]
# if debug:
#     eps_variants = [0.3]
#     min_samples_variants = [3]
# params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]
# 
# db_scan_results = {}
# db_scan_best_results = []
# db_scan_best_bic_results = []
# for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
# 
#     dbscan_localization = compare_dbscan_hyp_v2(
#         # phase_tag=phase_tag,
#         params=params,
#         df_flat=df_image_name,
#         # output_path=output_path,
#         output_plot_path=output_plot_path,
#         plot=show_plots,
#         
#     )
# 
#     db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
#     
#     # TODO Here lies the main problem with DBSCAN.
#     ## DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
#     ## Sillouette Scoring needs a minimum of 2 clusters
#     ## if there are points in decent radius they will belong to a cluster
#     # if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
#     #     db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
#     #     db_scan_best_bic_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
#     # # If two or more cluster seem to exists take ones with the best BIC or Silouette score
#     # else:  
#     # take the best result by silouette score if there are more clusters then 1
#     ## TODO make the sorting deterministic
#     db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_silouette_score", ascending=False).iloc[0])
#     
# df_dbscan_localization = pd.concat([*db_scan_results.values()])
# df_scan_best_results = pd.DataFrame(db_scan_best_results)



In [40]:
# df_scan_best_results

In [41]:
## fixes the problem with the silouette score sorting
from zooniverse.analysis import compare_dbscan_hyp_v2

eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
min_samples_variants = [3, 5, 8, 10]
if debug:
    eps_variants = [0.3]
    min_samples_variants = [3]
params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]

db_scan_results = {}
db_scan_best_results = []
db_scan_best_bic_results = []
for image_name, df_image_name in df_merged_dataset.groupby("image_name"):

    dbscan_localization = compare_dbscan_hyp_v2(
        # phase_tag=phase_tag,
        params=params,
        df_flat=df_image_name,
        # output_path=output_path,
        output_plot_path=output_plot_path,
        plot=show_plots,
        
    )

    db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
    
    # TODO Here lies the main problem with DBSCAN.
    # DBSCAN tends to classfy all points as noise if min_samples is too high. Often only a single user marked an iguana.
    # Sillouette Scoring needs a minimum of 2 clusters
    # if there are points in decent radius they will belong to a cluster
    if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        db_scan_best_bic_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
        # If two or more cluster seem to exists take ones with the best Silouette score
    else:  
        # take the best result by silouette score if there are more clusters then 1
        db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values(["dbscan_silouette_score", "dbscan_count"], ascending=[False, False]).iloc[0])
    
df_dbscan_localization = pd.concat([*db_scan_results.values()])
df_scan_best_results = pd.DataFrame(db_scan_best_results)



/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zoon

In [42]:
df_scan_best_results

,image_name,dbscan_count,dbscan_noise,dbscan_silouette_score,dbscan_BIC_score,eps,min_samples
12,EGI04-1_141.jpg,2,4,0.789230,-225.342799,0.20,3
0,EGI08-2_78.jpg,0,8,NaN,NaN,0.01,3
0,EPS01_28.jpg,0,9,NaN,NaN,0.01,3
8,EPS01_41.jpg,3,2,0.930901,-320.924560,0.10,3
24,EPS02_208.jpg,4,0,0.907422,-662.351013,0.50,3
...,...,...,...,...,...,...,...
12,PWC03-2-2_92.jpg,4,0,0.928263,-412.386216,0.20,3
8,PWC03-2-4_66.jpg,2,4,0.801725,-227.740439,0.10,3
24,PWC03-2-4_82.jpg,1,9,NaN,NaN,0.50,3
16,PWC03-2-4_85.jpg,2,0,0.967358,-112.159695,0.30,3


Here it can be seen why the silouette score is difficult because it is often undefined.

In [43]:
## save the combinations of parameters, which maximized the silouette score.

df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])
df_scan_best_results

,image_name,dbscan_count,dbscan_noise,dbscan_silouette_score,dbscan_BIC_score,eps,min_samples
12,EGI04-1_141.jpg,2,4,0.789230,-225.342799,0.20,3
0,EGI08-2_78.jpg,0,8,NaN,NaN,0.01,3
0,EPS01_28.jpg,0,9,NaN,NaN,0.01,3
8,EPS01_41.jpg,3,2,0.930901,-320.924560,0.10,3
24,EPS02_208.jpg,4,0,0.907422,-662.351013,0.50,3
...,...,...,...,...,...,...,...
12,PWC03-2-2_92.jpg,4,0,0.928263,-412.386216,0.20,3
8,PWC03-2-4_66.jpg,2,4,0.801725,-227.740439,0.10,3
24,PWC03-2-4_82.jpg,1,9,NaN,NaN,0.50,3
16,PWC03-2-4_85.jpg,2,0,0.967358,-112.159695,0.30,3


In [44]:
df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil" }, inplace=True)

df_comparison = df_comparison.merge(df_scan_best_results, on='image_name', how='left')

In [45]:
df_comparison.fillna(0, inplace=True)

mse_errors["dbscan_count_sil_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.dbscan_count_sil, squared=False)

pd.Series(mse_errors)

AttributeError: 'DataFrame' object has no attribute 'count_total'

In [46]:

df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil
0,EGI04-1_141.jpg,2.0,1.80,1,15,27,"[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4]",2
1,EGI08-2_78.jpg,1.0,1.00,1,8,8,"[1, 1, 1, 1, 1, 1, 1, 1]",0
2,EPS01_28.jpg,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]",0
3,EPS01_41.jpg,2.0,1.89,2,19,36,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",3
4,EPS02_208.jpg,2.0,2.59,2,27,70,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",4
...,...,...,...,...,...,...,...,...
1030,PWC03-2-2_92.jpg,3.5,3.12,4,16,50,"[1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]",4
1031,PWC03-2-4_66.jpg,2.0,1.85,2,13,24,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4]",2
1032,PWC03-2-4_82.jpg,1.0,1.00,1,16,16,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
1033,PWC03-2-4_85.jpg,1.0,1.25,1,12,15,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]",2


### HDBSCAN clustering for each image

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html#sklearn.cluster.HDBSCAN states: "A distance threshold. Clusters below this value will be merged."



In [47]:
from zooniverse.analysis import HDBSCAN_Wrapper

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("image_name"):
    annotations_count = get_mark_overview(df_image_name)
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"])
    
    # if less than min_cluster_sizes points are available clustering makes no sense
    if df_image_name.shape[0] >= 5: # If num_samples is 5 for the min_cluster_size is 5 there is no point in passing data with less than 5 samples
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = HDBSCAN_Wrapper(df_marks=df_image_name[["x", "y"]],
                                     annotations_count=annotations_count,
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zooniverse/zooniverse/utils/basic_clustering.py:128: RuntimeWarning: invalid value encountered in scalar divide
  variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
/Users/christian/PycharmProjects/iguanas-from-above-zoon

In [48]:
df_hdbscan.drop(["with_noise", "bic_avg"], axis=1, inplace=True)
df_hdbscan

,image_name,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,EGI04-1_141.jpg,2,0.0,5,None,3
0,EGI08-2_78.jpg,1,0.0,5,None,3
0,EPS01_28.jpg,1,0.0,5,None,4
0,EPS01_41.jpg,2,0.0,5,None,2
0,EPS02_208.jpg,5,0.0,5,None,0
...,...,...,...,...,...,...
0,PWC03-2-2_92.jpg,4,0.0,5,None,0
0,PWC03-2-4_66.jpg,2,0.0,5,None,0
0,PWC03-2-4_82.jpg,1,0.0,5,None,11
0,PWC03-2-4_85.jpg,1,0.0,5,None,10


In [49]:
df_comparison = df_comparison.merge(df_hdbscan, on='image_name', how='left')
df_comparison.fillna(0, inplace=True)
df_comparison

,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,EGI04-1_141.jpg,2.0,1.80,1,15,27,"[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4]",2,2,0.0,5,0,3
1,EGI08-2_78.jpg,1.0,1.00,1,8,8,"[1, 1, 1, 1, 1, 1, 1, 1]",0,1,0.0,5,0,3
2,EPS01_28.jpg,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]",0,1,0.0,5,0,4
3,EPS01_41.jpg,2.0,1.89,2,19,36,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",3,2,0.0,5,0,2
4,EPS02_208.jpg,2.0,2.59,2,27,70,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",4,5,0.0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,PWC03-2-2_92.jpg,3.5,3.12,4,16,50,"[1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]",4,4,0.0,5,0,0
1031,PWC03-2-4_66.jpg,2.0,1.85,2,13,24,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4]",2,2,0.0,5,0,0
1032,PWC03-2-4_82.jpg,1.0,1.00,1,16,16,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,1,0.0,5,0,11
1033,PWC03-2-4_85.jpg,1.0,1.25,1,12,15,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]",2,1,0.0,5,0,10


In [51]:
df_comparison[["mode_count", "HDBSCAN_count"]].sum()


mode_count       3331
HDBSCAN_count    3939
dtype: int64

In [52]:
df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

saved /Users/christian/data/zooniverse/2024_03_07_None_analysis/Iguanas 3rd launch/Iguanas 3rd launch_method_comparison.csv


In [53]:
df_comparison.count_total.fillna(0, inplace=True)
df_comparison.HDBSCAN_count.fillna(0, inplace=True)

mse_errors["hdbscan_count_rmse"] = mean_squared_error(df_comparison.count_total, df_comparison.HDBSCAN_count, squared=False)


AttributeError: 'DataFrame' object has no attribute 'count_total'

# A look into the results
Root Means Squared Error for the different methods

In [ ]:
df_rmse = pd.DataFrame(pd.Series(mse_errors).sort_values())

df_rmse.to_csv(config["rmse_errors"])
df_rmse

## The sum of the clustering
What is the sum of the methods

In [54]:

df_comparison_sum = df_comparison[["count_total", "median_count", "mean_count", "mode_count", "dbscan_count_sil", "HDBSCAN_count"]].sum().sort_values()
df_comparison_sum.to_csv(config["method_sums"])
df_comparison_sum

KeyError: "['count_total'] not in index"

In [55]:
print(f"phase_tag: {phase_tag}, user_threshold: {user_threshold}")

phase_tag: Iguanas 3rd launch, user_threshold: 5


## Compare the numbers
The counts are only for images which were in the dataset after filtering.

### Sum of all the Methods

In [ ]:
print(f"{config['method_sums'].name}")
pd.read_csv(config["method_sums"])

### Root Mean Squared Error

In [ ]:
print(f"{config['rmse_errors'].name}")
pd.read_csv(config["rmse_errors"])

### Comparison per Image Level

In [56]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])

load /Users/christian/data/zooniverse/2024_03_07_None_analysis/Iguanas 3rd launch/Iguanas 3rd launch_method_comparison.csv


,Unnamed: 0,image_name,median_count,mean_count,mode_count,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,0,EGI04-1_141.jpg,2.0,1.80,1,15,27,"[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 4]",2,2,0.0,5,0,3
1,1,EGI08-2_78.jpg,1.0,1.00,1,8,8,"[1, 1, 1, 1, 1, 1, 1, 1]",0,1,0.0,5,0,3
2,2,EPS01_28.jpg,1.0,1.00,1,9,9,"[1, 1, 1, 1, 1, 1, 1, 1, 1]",0,1,0.0,5,0,4
3,3,EPS01_41.jpg,2.0,1.89,2,19,36,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",3,2,0.0,5,0,2
4,4,EPS02_208.jpg,2.0,2.59,2,27,70,"[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",4,5,0.0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,1030,PWC03-2-2_92.jpg,3.5,3.12,4,16,50,"[1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]",4,4,0.0,5,0,0
1031,1031,PWC03-2-4_66.jpg,2.0,1.85,2,13,24,"[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 4]",2,2,0.0,5,0,0
1032,1032,PWC03-2-4_82.jpg,1.0,1.00,1,16,16,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,1,0.0,5,0,11
1033,1033,PWC03-2-4_85.jpg,1.0,1.25,1,12,15,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2]",2,1,0.0,5,0,10


## Discussion:
Clustering works, it yields better numbers than just taking mode,median or mean annotations from the volunteers, because it takes the spatial location of the marker dots into consideration.



Assert the numbers haven't changed

In [ ]:
## these are the numbers before the sorting was repaired
# if phase_tag == "Iguanas 1st launch" and not debug:
#     if user_threshold == 3:
#         assert df_comparison_sum["mode_count"] == 215
#         assert df_comparison_sum["dbscan_count_sil"] == 222
#         assert df_comparison_sum["median_count"] == 228.5
#         assert df_comparison_sum["HDBSCAN_count"] == 244
#         assert df_comparison_sum["count_total"] == 323
#     if user_threshold is None:
#         assert df_comparison_sum["mode_count"] == 221
#         assert df_comparison_sum["dbscan_count_sil"] == 224
#         assert df_comparison_sum["median_count"] == 235.5
#         assert df_comparison_sum["HDBSCAN_count"] == 247
#         assert df_comparison_sum["count_total"] == 331
#         
# if phase_tag == "Iguanas 2nd launch" and not debug:
#     if user_threshold == 3:
#         assert df_comparison_sum["mode_count"] == 502
#         assert df_comparison_sum["dbscan_count_sil"] == 484
#         assert df_comparison_sum["median_count"] == 475
#         assert df_comparison_sum["HDBSCAN_count"] == 541
#         assert df_comparison_sum["count_total"] == 586
#     if user_threshold is None:
#         assert df_comparison_sum["mode_count"] == 511
#         assert df_comparison_sum["dbscan_count_sil"] == 484
#         assert df_comparison_sum["median_count"] == 484.5
#         assert df_comparison_sum["HDBSCAN_count"] == 541.0
#         assert df_comparison_sum["count_total"] == 589.0
#         
# if phase_tag == "Iguanas 3rd launch" and not debug:
#     if user_threshold == 3:
#         assert df_comparison_sum["mode_count"] == 302
#         assert df_comparison_sum["dbscan_count_sil"] == 309
#         assert df_comparison_sum["median_count"] == 313
#         assert df_comparison_sum["HDBSCAN_count"] == 357
#         assert df_comparison_sum["count_total"] == 351
#     if user_threshold is None:
#         assert df_comparison_sum["mode_count"] == 304
#         assert df_comparison_sum["dbscan_count_sil"] == 309
#         assert df_comparison_sum["median_count"] == 315
#         assert df_comparison_sum["HDBSCAN_count"] == 357
#         assert df_comparison_sum["count_total"] == 351
